In [1]:
from optimum.intel.openvino import OVModelForSpeechSeq2Seq
from transformers import AutoProcessor

model_id = "openai/whisper-tiny"

from pathlib import Path

model_dir = model_id.split("/")[-1]

if not Path(model_dir).exists():
    !optimum-cli export openvino -m {model_id} {model_dir} --weight-format fp16

In [2]:
import openvino as ov

core = ov.Core()

In [3]:
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)
open("notebook_utils.py", "w").write(r.text)

from notebook_utils import device_widget

device = device_widget()

device

[ERROR] 08:06:08.786 [NPUBackends] Cannot find backend for inference. Make sure the device is available.


Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

In [9]:
from optimum.intel.openvino import OVModelForSpeechSeq2Seq
from transformers import AutoProcessor, pipeline

ov_model = OVModelForSpeechSeq2Seq.from_pretrained(model_dir, device=device.value)

processor = AutoProcessor.from_pretrained(model_dir)

pipe = pipeline(
    "automatic-speech-recognition",
    model=ov_model,
    chunk_length_s=30,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
)

Compiling the encoder to AUTO ...


[ERROR] 08:06:53.631 [NPUBackends] Cannot find backend for inference. Make sure the device is available.


Compiling the decoder to AUTO ...
Compiling the decoder to AUTO ...


In [4]:
import ipywidgets as widgets

VIDEO_LINK = "https://youtu.be/kgL5LBM-hFI"
link = widgets.Text(
    value=VIDEO_LINK,
    placeholder="Type link for video",
    description="Video:",
    disabled=False,
)

link

Text(value='https://youtu.be/kgL5LBM-hFI', description='Video:', placeholder='Type link for video')

In [5]:
from pathlib import Path
import yt_dlp

print(f"Downloading video {link.value} started")

output_file = Path("downloaded_video.mp4")
ydl_ops = {"format": "best[ext=mp4]", "outtmpl": output_file.as_posix()}
with yt_dlp.YoutubeDL(ydl_ops) as ydl:
    ydl.download(link.value)

print(f"Video saved to {output_file}")

[youtube] Extracting URL: https://youtu.be/kgL5LBM-hFI
[youtube] kgL5LBM-hFI: Downloading webpage
[youtube] kgL5LBM-hFI: Downloading ios player API JSON
[youtube] kgL5LBM-hFI: Downloading web creator player API JSON
[youtube] kgL5LBM-hFI: Downloading player b0557ce3
[youtube] kgL5LBM-hFI: Downloading m3u8 information
[info] kgL5LBM-hFI: Downloading 1 format(s): 18
[download] Destination: downloaded_video.mp4
[download] 100% of    1.20MiB in 00:00:01 at 722.63KiB/s 
Video saved to downloaded_video.mp4


In [6]:
task = widgets.Select(
    options=["transcribe", "translate"],
    value="translate",
    description="Select task:",
    disabled=False,
)
task

Select(description='Select task:', index=1, options=('transcribe', 'translate'), value='translate')

In [7]:
from moviepy.editor import VideoFileClip
from transformers.pipelines.audio_utils import ffmpeg_read


def get_audio(video_file):
    """
    Extract audio signal from a given video file, then convert it to float,
    then mono-channel format and resample it to the expected sample rate

    Parameters:
        video_file: path to input video file
    Returns:
      resampled_audio: mono-channel float audio signal with 16000 Hz sample rate
                       extracted from video
      duration: duration of video fragment in seconds
    """
    input_video = VideoFileClip(str(video_file))
    duration = input_video.duration
    audio_file = video_file.stem + ".wav"
    input_video.audio.write_audiofile(audio_file, verbose=False, logger=None)
    with open(audio_file, "rb") as f:
        inputs = f.read()
    audio = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
    return {
        "raw": audio,
        "sampling_rate": pipe.feature_extractor.sampling_rate,
    }, duration

In [10]:
inputs, duration = get_audio(output_file)

transcription = pipe(inputs, generate_kwargs={"task": task.value}, return_timestamps=True)["chunks"]

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
